## From The standard Node-Link data to Network
Data Source : http://nodelink.its.go.kr/data/data01.aspx

The standard node-link files consist of two kinds of shape files(node, link).

The _shapefile_ format is a popular geospatial vector data format for geographic information system (GIS) software. It is developed and regulated by Esri as a (mostly) open specification for data interoperability among Esri and other GIS software products. [wiki](https://en.wikipedia.org/wiki/Shapefile)

To construct network, we should modifiy shpfiles.

In [1]:
import shapefile #the pyshp module : Should install pyshp module.
import pandas as pd
from pyproj import Proj, transform # Should install pyproj module.
import networkx as nx
import folium

If you see an error message that `No module named modulename`, you should install module. To resolve this problem, we need to istall 'pyshp' and 'pyproj'. To do this, we use `pip`. We can install module easily by typing `pip install modulename`. To install `pyshp` and `pyproj`, we type the below codes in `terminal` or `anaconda prompt`.

```
pip install pyshp
pip install pyproj
```
If there is a problem to install pyproj, then do the following; conda install -c anaconda pyproj 

## Construct dataframe by using shp file

In [3]:
# read data (Copy all files from nodelink into nodelink folder: I made it.)
# using old_data
shp_path_node = './HOUSPOP/XsDB_주거인구_100M_TM.shp'
sf_node = shapefile.Reader(shp_path_node)

In [4]:
# construct pandas dataframe

#grab the shapefile's field names
# node
fields_node = [x[0] for x in sf_node.fields][1:]
records_node = sf_node.records()
shps = [s.points for s in sf_node.shapes()] # node has coordinate data.

#write the records into a dataframe
# node
node_dataframe = pd.DataFrame(columns=fields_node, data=records_node)
#add the coordinate data to a column called "coords"
node_dataframe = node_dataframe.assign(coords=shps)
# link
# link_dataframe = pd.DataFrame(columns=fields_link, data=records_link)

''' Show the files'''

In [4]:
node_dataframe.head()# Show first 5 items.

,NM,X_AXIS,Y_AXIS,b'\xb1\xb8\xba\xd0',b'\xb9\xe8\xc6\xf7\xc3\xb3',coords
0,b'\xb0\xa1\xbe\xe7\xc0\xcc\xb8\xb6\xc6\xae\xc1...,299675.0,551244.0,b'\xb9\xf6\xb0\xc5\xc5\xb7',b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[299675.0, 551244.0]]"
1,b'\xb0\xad\xb3\xb2\xb1\xb3\xba\xb8\xc1\xa1',313859.0,545186.0,b'\xb9\xf6\xb0\xc5\xc5\xb7',b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[313859.0, 545186.0]]"
2,b'\xb0\xad\xb3\xb2\xb1\xb8\xc3\xbb\xc1\xa1',315532.0,546133.0,b'\xb9\xf6\xb0\xc5\xc5\xb7',b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[315532.0, 546133.0]]"
3,b'\xb0\xad\xb3\xb2\xb4\xeb\xb7\xce\xc1\xa1',314226.0,544524.0,b'\xb9\xf6\xb0\xc5\xc5\xb7',b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[314226.0, 544524.0]]"
4,b'\xb0\xad\xb3\xb2\xbf\xaa\xc1\xa1',314020.0,544519.0,b'\xb9\xf6\xb0\xc5\xc5\xb7',b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[314020.0, 544519.0]]"


In [5]:
col = list(node_dataframe.columns)

In [6]:
for i in range(len(col)):
    if(type(col[i])==bytes):
        col[i] = col[i].decode('cp949')
node_dataframe.columns = col

In [7]:
node_dataframe['구분'] = node_dataframe['구분'].apply(lambda x : x.decode('cp949'))

In [8]:
nm = []
for i in node_dataframe['NM']:
    if (type(i) == bytes):
        nm.append(i.decode('cp949'))
    else:
        nm.append(i)
node_dataframe['NM'] = nm       

In [9]:
node_dataframe

,NM,X_AXIS,Y_AXIS,구분,배포처,coords
0,가양이마트점,299675.000000,551244.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[299675.0, 551244.0]]"
1,강남교보점,313859.000000,545186.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[313859.0, 545186.0]]"
2,강남구청점,315532.000000,546133.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[315532.0, 546133.0]]"
3,강남대로점,314226.000000,544524.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[314226.0, 544524.0]]"
4,강남역점,314020.000000,544519.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[314020.0, 544519.0]]"
5,강남진솔점,314354.000000,544170.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[314354.0, 544170.0]]"
6,강남NC점,312434.000000,545709.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[312434.0, 545709.0]]"
7,건대역점,318232.000000,548803.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[318232.0, 548803.0]]"
8,관악롯데점,305139.000000,543696.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[305139.0, 543696.0]]"
9,광명소하이마트점,301475.000000,538852.000000,버거킹,b'\xb9\xe8\xc6\xf7:www.biz-gis.com',"[[301475.0, 538852.0]]"


<img src="http://www.biz-gis.com/files/attach/images/5130/326/067/603e3a2fa9d2bd201deeda6df2d8c402.jpg"></img>

In nodelink data, all positions in nodes are assigned based on **korea 2000 좌표계**. Their positions are changed based on 
**wgs84 (위도/경도)** by using Proj package.

In [10]:
x = node_dataframe['coords'][0]

In [11]:
x[0][0]

299675.0

In [12]:
x[0][1]

551244.0

In [166]:
# inProj = Proj(init = 'epsg:2097')
inProj = Proj(init = 'epsg:5188')
outProj= Proj(init = 'epsg:4326')

In [167]:
transform(inProj, outProj, x[0][0], x[0][1])

(132.12803488229048, 37.55533742982708)

In [168]:
# Change coordinate system
# korea 2000/central belt 2010 (epsg:5186) to wgs84(epsg:4326)
node_dataframe['latitude'] = node_dataframe['coords'].apply(lambda x: transform(inProj, outProj, x[0][0], x[0][1])[1])
node_dataframe['longitude'] = node_dataframe['coords'].apply(lambda x: transform(inProj, outProj, x[0][0], x[0][1])[0])

In order to use Gephi, it is essential to have two files node and line file. Also each one has a special properties. That is, node name has to be indexed as an **ID** and link has two names **Source** and **Target**. File in below show that how to change the given form to gephi-type file form.

In [169]:
column_list = list(node_dataframe.columns)
column_list[-4] = column_list[-4].decode('cp949')

AttributeError: 'str' object has no attribute 'decode'

In [170]:
node_dataframe.columns = column_list

In [171]:
data = node_dataframe.drop(['X_AXIS','Y_AXIS','coords','배포처'], axis=1)
data.head(10)

,NM,구분,latitude,longitude
0,가양이마트점,버거킹,37.555337,132.128035
1,강남교보점,버거킹,37.499126,132.287584
2,강남구청점,버거킹,37.507449,132.306647
3,강남대로점,버거킹,37.493118,132.291630
4,강남역점,버거킹,37.493098,132.289301
5,강남진솔점,버거킹,37.489913,132.293022
6,강남NC점,버거킹,37.504012,132.271553
7,건대역점,버거킹,37.531157,132.337606
8,관악롯데점,버거킹,37.486737,132.188780
9,광명소하이마트점,버거킹,37.443511,132.146693


In [172]:
incheon_data = data[data['MEGA_NM'] == '인천광역시']
len(incheon_data)

KeyError: 'MEGA_NM'

In [173]:
seoul_data = data[data['MEGA_NM'] == '서울특별시']
len(seoul_data)

KeyError: 'MEGA_NM'

In [174]:
ulsan_data = data[data['MEGA_NM'] == '울산광역시']
len(ulsan_data)

KeyError: 'MEGA_NM'

In [175]:
ulsan_data.head(10)

NameError: name 'ulsan_data' is not defined

In [176]:
sorted_ulsan = ulsan_data.sort_values(by=['POP'],ascending=False)
sorted_ulsan.head(10)

NameError: name 'ulsan_data' is not defined

## Data Verification

In [177]:
import folium

In [178]:
loc = tuple(sorted_ulsan.iloc[0][['latitude','longitude']])
loc

NameError: name 'sorted_ulsan' is not defined

In [179]:

map_osm = folium.Map(location=loc, zoom_start=13)

for i in range(3):
    location = tuple(sorted_ulsan.iloc[i][['latitude','longitude']])
    folium.Marker(location,popup=str(int(sorted_ulsan.iloc[i]['POP']))).add_to(map_osm)
map_osm

NameError: name 'loc' is not defined

In [180]:
data = node_dataframe.drop(['X_AXIS','Y_AXIS','coords','배포처'], axis=1)
data.head(10)

,NM,구분,latitude,longitude
0,가양이마트점,버거킹,37.555337,132.128035
1,강남교보점,버거킹,37.499126,132.287584
2,강남구청점,버거킹,37.507449,132.306647
3,강남대로점,버거킹,37.493118,132.291630
4,강남역점,버거킹,37.493098,132.289301
5,강남진솔점,버거킹,37.489913,132.293022
6,강남NC점,버거킹,37.504012,132.271553
7,건대역점,버거킹,37.531157,132.337606
8,관악롯데점,버거킹,37.486737,132.188780
9,광명소하이마트점,버거킹,37.443511,132.146693


In [181]:

map_osm = folium.Map(location=(38.393846,128.305295), zoom_start=13)

for i in range(100):
    location = tuple(data.iloc[i][['latitude','longitude']])
    folium.Marker(location,popup=data.iloc[i]['구분']).add_to(map_osm)
map_osm

이상하네 이 데이터는 쓰면 안되겠다 좌표계 인코딩이 문제인듯

# You can save Data as csv formet.

In [243]:
ulsan_data.to_csv('Ulsan_population.csv', encoding='cp949') 